

{% include toc title="In This Lesson" icon="file-text" %}



<div class='notice--success' markdown="1">



## <i class="fa fa-graduation-cap" aria-hidden="true"></i> Learning Objectives



After completing this tutorial, you will be able to:



* Open MODIS imagery in R

* Create NBR index using MODIS imagery in R

* Calculate total burned area in R.



## <i class="fa fa-check-square-o fa-2" aria-hidden="true"></i> What you need



You will need a computer with internet access to complete this lesson and the

data for week 6 of the course.



{% include/data_subsets/course_earth_analytics/_data-week6-7.md %}

</div>






In [1]:
```{r load-libraries, echo=F, results='hide', message=F, warning=F}

library(raster)

library(rgeos)

library(rgdal)

options(stringsAsFactors=F)

# import vector that we used to crop the data

# csf_crop <- readOGR("data/week6/vector_layers/fire_crop_box_500m.shp")



```




First, let's import MODIS data. Below notice that we have used a slightly different

version of the `list.files()` `pattern=` argument.



We have used `glob2rx("*sur_refl*.tif$")` to select all layers that both



1. Contain the word `sur_refl` in them and

2. Contain the extension `.tif`



Let's import our MODIS image stack.




In [1]:
```{r work-with-modis}

# open modis bands (layers with sur_refl in the name)

all_modis_bands_july7 <-list.files("data/week6/modis/reflectance/07_july_2016/crop",

           pattern=glob2rx("*sur_refl*.tif$"),

           full.names = T)

# create spatial raster stack

all_modis_bands_st_july7 <- stack(all_modis_bands_july7)



# view range of values in stack

all_modis_bands_st_july7[[2]]



# view band names

names(all_modis_bands_st_july7)

# clean up the band names for neater plotting

names(all_modis_bands_st_july7) <- gsub("MOD09GA.A2016189.h09v05.006.2016191073856_sur_refl_b", "Band",

     names(all_modis_bands_st_july7))



# view cleaned up band names

names(all_modis_bands_st_july7)

```






## Reflectance values range 0-1



As we've discussed in class, the normal range of reflectance values is 0-1 where

1 is the BRIGHTEST values and 0 is the darkest value. Have a close look at the

min and max values in the second raster layer of our stack, above. What do you notice?



The min and max values are widely outside of the expected range of 0-1  - min: -32768, max: 32767

What could be causing this? We need to better understand our data before we can

work with it more. Have a look at the table in the MODIS users guide. The data

that we are working with is the MOD09GA product. Look closely at the table on

page 14 of the guide. Part of the table can be seen below.



<a href="http://modis-sr.ltdri.org/guide/MOD09_UserGuide_v1_3.pdf" target="_blank">Click here to check out the MODIS user guide - check out page 14 for the MOD09GA table.</a>



The column headings for the table below:



| Group | Science Data Sets (HDF Layers (21)) | Units | Data Type | Fill Value | Valid Range | Scale Factor |

|---|

| | surf_Refl_b01: 500m Surface Reflectance Band 1 (620-670 nm) | Reflectance | 16-bit signed integer | -28672 | -100 - 16000 | 0.0001 |

| | surf_Refl_b02: 500m Surface Reflectance Band 2 (841-876 nm) | Reflectance | 16-bit signed integer  | -28672 | -100 - 16000 | 0.0001 |

| | surf_Refl_b03: 500m Surface Reflectance Band 3 (459-479 nm)| Reflectance | 16-bit signed integer  | -28672 | -100 - 16000 | 0.0001 |

| | surf_Refl_b04: 500m Surface Reflectance Band 4 (545-565 nm)| Reflectance | 16-bit signed integer  | -28672 | -100 - 16000 | 0.0001 |

| | surf_Refl_b05: 500m Surface Reflectance Band 5 (1230-1250 nm)| Reflectance | 16-bit signed integer  | -28672 | -100 - 16000 | 0.0001 |

| | surf_Refl_b06: 500m Surface Reflectance Band 6 (1628-1652 nm) | Reflectance | 16-bit signed integer  | -28672 | -100 - 16000 | 0.0001 |

| | surf_Refl_b07: 500m Surface Reflectance Band 7 (2105-2155 nm) | Reflectance | 16-bit signed integer  | -28672 | -100 - 16000 | 0.0001 |



<figure>

<a href="{{ site.url }}/images/course-materials/earth-analytics/week-7/MOD09GA-metadata.png" target="_blank">

   <img src="{{ site.url }}/images/course-materials/earth-analytics/week-7/MOD09GA-metadata.png" alt="MODIS MOD09GA metadata"></a>

   <figcaption>Notice the valid values for the MOD09GA reflectance product. The range

   is -100 to 16000.

   </figcaption>

</figure>



Looking at the table, answer the following questions



1. What is valid range of values for our data?

2. What is the scale factor associated with our data?



## Explore our data



Looking at histograms of our data, we can see that the range of values is not

what we'd expect. We'd expect values between -100 to 10000 yet instead we have

much larger numbers.








In [1]:
```{r explore-data, warning=F, message=F, fig.cap="MODIS stack band 2 plot"}

# turn off scientific notation

options("scipen"=100, "digits"=4)

# bottom, left, top and right

#par(mfrow=c(4, 2))

hist(all_modis_bands_st_july7,

  col="springgreen",

  xlab="Reflectance Value")

mtext("Distribution of MODIS reflectance values for each band\n Data not scaled", 

      outer = TRUE, cex = 1.5)



```


In [1]:
```{r echo=F, results='hide'}

dev.off()

```




## Scale Factor

Looking at the metadata, we can see that our  data have a scale factor. Let's

deal with that before doing anything else. The scale factor is .0001. This means

we should multiple each layer by that value to get the actual reflectance values

of the data.



We can apply this math to all of the layers in our stack using a simple calculation

shown below:




In [1]:
```{r scale-data, fig.cap="MODIS stack histogram plot", fig.width=7, fig.height=8}

# deal with nodata value --  -28672

all_modis_bands_st_july7 <- all_modis_bands_st_july7 * .0001

# view histogram of each layer in our stack

# par(mfrow=c(4, 2))

hist(all_modis_bands_st_july7,

   xlab="Reflectance Value",

   col="springgreen")

mtext("Distribution of MODIS reflectance values for each band\n Scale factor applied", outer = TRUE, cex = 1.5)

```




Great - now the range of values in our data appear more reasonable. Next, let's

get rid of data that are outside of the valid data range.



## NoData Values



Next, let's deal with no data values. We can see that our data have a "fill" value

of -28672 which we can presume to be missing data. But also we see that valid

range values begin at -100. Let's set all values less than -100 to NA to remove

the extreme negative values that may impact out analysis.




In [1]:
```{r echo=F, results='hide'}

dev.off()

```


In [1]:
```{r assign-no-data, fig.cap="MODIS stack histogram plot with NA removed"}

# deal with nodata value --  -28672

all_modis_bands_st_july7[all_modis_bands_st_july7 < -100 ] <- NA

#par(mfrow=c(4,2))

# plot histogram

hist(all_modis_bands_st_july7,

  xlab="Reflectance Value",

  col="springgreen")

mtext("Distribution of reflectance values for each band", outer = TRUE, cex = 1.5)

```


In [1]:
```{r echo=F, results='hide'}

dev.off()

```




Next we plot MODIS layers. Use the MODIS band chart to figure out what bands you

need to plot to create a RGB (true color) image.



| Band | Wavelength range (nm) | Spatial Resolution (m) | Spectral Width (nm)|

|-------------------------------------|------------------|--------------------|----------------|

| Band 1 - red | 620 - 670 | 250 | 2.0 |

| Band 2 - near infrared | 841 - 876 | 250 | 6.0 |

| Band 3 -  blue/green | 459 - 479 | 500 | 6.0 |

| Band 4 - green | 545 - 565 | 500 | 3.0 |

| Band 5 - near infrared  | 1230 – 1250 | 500 | 8.0  |

| Band 6 - mid-infrared | 1628 – 1652 | 500 | 18 |

| Band 7 - mid-infrared | 2105 - 2155 | 500 | 18 |



In the plot below, i've called attention to the AOI boundary with a yellow color.

Why is it so hard to figure out where the study area is in this MODIS image?




In [1]:
```{r import-shapefile, results='hide', echo=F}

# view fire overlay boundary

fire_boundary <- readOGR("data/week6/vector_layers/fire-boundary-geomac/co_cold_springs_20160711_2200_dd83.shp")

fire_boundary_sin <- spTransform(fire_boundary,

                                 CRS=crs(all_modis_bands_st_july7))

# export as sinusoidal

# writeOGR(fire_boundary_sin,

#          dsn = "data/week6/vector_layers/fire-boundary-geomac",

#          layer="co_cold_springs_20160711_2200_sin",

#          driver="ESRI Shapefile",

#          overwrite_layer=TRUE)

```


In [1]:
```{r plot-modis-layers, echo=F, fig.cap="plot MODIS stack", fig.width=5, fig.height=5}

## 3 = blue, 4 = green, 1= red 2= nir

par(col.axis="white", col.lab="white", tck=0)

plotRGB(all_modis_bands_st_july7,

        r=1, g =4, b=3,

        stretch="lin",

        main="MODIS post-fire RGB image\n Cold springs fire site",

        axes=T)

box(col="white")

# add fire boundary to plot

plot(fire_boundary_sin,

     add=T,

     border="yellow",

     lwd=50)



```




## MODIS cloud mask



Next, we can deal with clouds in the same way that we dealt with them using

Landsat data. However, our cloud mask in this case is slightly different with

slightly different cloud cover values as follows:



| State | Translated Value | Cloud Condition|

|----|

| 00 | 0 | clear |

| 01 | 1 | cloudy |

| 10 | 2 | mixed |

| 11 | 3 | not set, assumed clear |



The metadata for the MODIS data are a bit trickier to figure out. If you are interested,

the link to the MODIS user guide is below.



* <a href="http://modis-sr.ltdri.org/guide/MOD09_UserGuide_v1_3.pdf" target="_blank">MODIS user guide</a>



The MODIS data are also stored natively in a H4 format which we will not be discussing

in this class. For the purposes of this assignment, use the table above to assign

cloud cover "values" and to create a mask.



Use the cloud cover layer `data/week6/modis/reflectance/07_july_2016/crop/cloud_mask_july7_500m`

to create your mask.



Set all values >0 in the cloud cover layer to `NA`.




In [1]:
```{r reset-dev, warning=F, echo=F, message=F, results='hide'}

dev.off()

```


In [1]:
```{r create-apply-mask, echo=F, fig.cap="cloud mask plot"}

# import cloud mask

cloud_mask_7July <- raster("data/week6/modis/reflectance/07_july_2016/crop/cloud_mask_july7_500m.tif")

cloud_mask_7July[cloud_mask_7July > 0] <- NA

plot(cloud_mask_7July,

     main="Landsat cloud mask layer",

     legend=F,

     axes=F, box=F)

legend('topright',

       legend=c("Cloud free", "Clouds"),

       fill=c("yellow", "white"))

```


In [1]:
```{r create-mask, fig.cap="Final stack masked", echo=F}

all_modis_bands_st_mask <- mask(all_modis_bands_st_july7,

                                cloud_mask_7July)



## 3 = blue, 4 = green, 1= red 2= nir

```




Plot the masked data. Notice that now the clouds are gone as they have been assigned

the value `NA`.




In [1]:
```{r masked-data, echo=F, fig.cap="MODIS with cloud mask", fig.width=7, fig.height=4}

## 3 = blue, 4 = green, 1= red 2= nir

par(col.axis="white", col.lab="white", tck=0)

plotRGB(all_modis_bands_st_mask,

        r=1, g =4, b=3,

        stretch="lin",

        main="MODIS data mask applied\n Cold springs fire AOI",

        axes=T)

box(col="white")

plot(fire_boundary_sin,

     add=T, col="yellow",

     lwd=1)

```




Finally crop the data to see just the pixels that overlay our study area.




In [1]:
```{r crop-data, echo=F, fig.cap="cropped data"}

all_modis_bands_st_mask <- crop(all_modis_bands_st_mask, fire_boundary_sin)

par(col.axis="white", col.lab="white", tck=0)

plotRGB(all_modis_bands_st_mask,

        r=1, g =4, b=3,

        stretch="lin",

        ext=extent(fire_boundary_sin),

        axes=T,

        main="Final landsat masked data \n Cold Springs fire scar site")

box(col="white")

plot(fire_boundary_sin, border="yellow", add=T)



```






| SEVERITY LEVEL  | NBR RANGE |

|------------------------------|

| Enhanced Regrowth | -700 to  -100  |

| Unburned       |  -100 to +100  |

| Low Severity     | +100 to +270  |

| Moderate Severity  | +270 to +660  |

| High Severity     |  +660 to +1300 |








In [1]:
```{r create-apply-mask2, echo=F, fig.cap="Classified pre fire NBR", fig.width=6, fig.height=4}

# Band 4 includes wavelengths from 0.76-0.90 µm (NIR) and

# Band 7 includes wavelengths between 2.09-2.35 µm (SWIR).

# B2 - B7 / b2 + b7

get_veg_index <- function(band1, band2){

  # calculate index

  index <- (band1 - band2) / (band1 + band2)

  return(index)

}



# calculate modis NBR

modis_nbr <- overlay(all_modis_bands_st_mask[[2]], all_modis_bands_st_mask[[7]],

                     fun=get_veg_index)

modis_nbr <- modis_nbr * 1000



# create classification matrix

reclass <- c(-1001, -100, 1,

             -100, 100, 2,

             100, 270, 3,

             270, 660, 4,

             660, 1500, 5)

# reshape the object into a matrix with columns and rows

reclass_m <- matrix(reclass,

                ncol=3,

                byrow=TRUE)



modis_nbr_cl <- reclassify(modis_nbr,

                     reclass_m)

# reclass data

plot(modis_nbr_cl,

     main="MODIS NBR for the Cold Springs site",

     legend=F,

     axes=F, box=F)

plot(fire_boundary_sin,

     add=T)

legend("topright",

       legend=c("Moderate burn \nseverity"),

       fill="yellow")



```




After we've calculated NBR, we may want to calculate total burn AREA. We can do

this using the `freq()` function in R. This function gives us the total number

of pixels associated with each value in our classified raster.



1. **Calculate frequency ignoring NA values:** `freq(modis_nbr_cl, useNA='no')`

2. **Calculate frequency, ignore NA & only include values that equal 5:** `freq(modis_nbr_cl, useNA='no', value=5)`



Let's use the MODIS data from 7 July 2016 to calculate the total area of land

classified as:



1. Burn: moderate severity

2. Burn: high severity






In [1]:
```{r get-pixel-values }

# get summary counts of each class in raster

freq(modis_nbr_cl, useNA='no')



final_burn_area_high_sev <- freq(modis_nbr_cl, useNA='no', value=5)

final_burn_area_moderate_sev <- freq(modis_nbr_cl, useNA='no', value=4)



```


In [1]:
```{r open-post-fire, echo=F}



# open modis bands (layers with sur_refl in the name)

all_modis_bands_july17 <-list.files("data/week6/modis/reflectance/17_july_2016/crop",

           pattern=glob2rx("*sur_refl*.tif$"),

           full.names = T)



all_modis_bands_st_july17 <- stack(all_modis_bands_july17)

# rescale data

all_modis_bands_st_july17 <- all_modis_bands_st_july17 * .0001



# deal with nodata value --  -28672

all_modis_bands_st_july17[all_modis_bands_st_july17 < -100] <- NA



# import cloud mask & Mask data

cloud_mask_17July <- raster("data/week6/modis/reflectance/17_july_2016/crop/cloud_mask_july17_500m.tif")

cloud_mask_17July[cloud_mask_17July > 0] <- NA

all_modis_bands_st_mask_july17 <- mask(all_modis_bands_st_july17,

                                cloud_mask_17July)



```




We can perform the steps that we performed above, on the MODIS post-fire data

too. Below is a plot of the July 17 data.




In [1]:
```{r plot-rgb-post-fire, fig.cap="RGB post fire", fig.height=5, fig.width=5}

par(col.axis="white", col.lab="white", tck=0)

# clouds removed

plotRGB(all_modis_bands_st_mask_july17,

        1,4,3,

        stretch="lin",

        main="Final data plotted with mask\n Post Fire - 17 July 2016",

        axes=T)

box(col="white")

```




Next we calculate NBR on our post fire data. Then we can crop and finally

plot the final results!




In [1]:
```{r mask-data, echo=F}

# calculate NBR

modis_nbr_july17 <- overlay(all_modis_bands_st_mask_july17[[2]],

                            all_modis_bands_st_mask_july17[[7]],

                            fun=get_veg_index)



modis_nbr_july17 <- modis_nbr_july17 * 1000



modis_nbr_july17_cl <- reclassify(modis_nbr_july17,

                     reclass_m)

# crop to final extent



modis_nbr_july17_cl <- crop(modis_nbr_july17_cl, fire_boundary_sin)

```




## Post fire NBR results






In [1]:
```{r view-barplot, fig.cap="barplot of final post fire classified data.", fig.height=5, fig.width=5}

the_colors = c("palevioletred4","palevioletred1","ivory1")

barplot(modis_nbr_july17_cl,

        main="Distribution of burn values - Post Fire",

        col=rev(the_colors),

        names.arg=c("Low Severity","Moderate Severity","High Severity"))

```






Finally, plot the reclassified data. Note that we only have 3 classes: 2, 3 and 4.






In [1]:
```{r plot-data-reclass, echo=F, fig.width=6, fig.height=4, fig.cap="MODIS NBR plot w colors"}

# the_colors = c("palevioletred4","palevioletred1","ivory1","seagreen1","seagreen4")

the_colors = c("ivory1","palevioletred1","palevioletred4")



# reclass data

# mar bottom, left, top and right

par(xpd = F, mar=c(0,0,2,5))

plot(modis_nbr_july17_cl,

     main="MODIS NBR for the Cold Springs site \n Post fire",

     ext=extent(fire_boundary_sin),

     col=the_colors,

     axes=F,

     box=F,

     legend=F)

plot(fire_boundary_sin,

     add=T)

par(xpd = TRUE)

legend(modis_nbr_july17_cl@extent@xmax-50, modis_nbr_july17_cl@extent@ymax,

       c("Low Severity", "Moderate Severity", "High Severity"),

       fill=the_colors,

       cex=.8,

       bty="n")



final_burn_area_high_sev <- freq(modis_nbr_july17_cl, useNA='no', value=5)

final_burn_area_moderate_sev <- freq(modis_nbr_july17_cl, useNA='no', value=4)

```


In [1]:
```{r dev-off, echo=F, results='hide'}

dev.off()

```
